### General description

This notebook shows an example of how interactive OEMOF system graphs can be created with [Plotly-Dash](https://dash.plotly.com/) for defined energy systems.

Three functions are defined for this purpose: 

1. the `make_network` function creates a directed graph (`DiGraph`) from the `oemof` energy system. The nodes of the graph represent the components of the energy system, and the edges represent the connections between these components. Based on the type of the components (source, sink, converter, bus, generic memory), colors and shapes are assigned to the nodes.

2. the `make_cytoscape_elements` function converts the NetworkX graph into Cytoscape elements that can be used for visualization with Dash. Each node in the graph receives a dictionary with its properties such as ID, label, color and shape. The edges of the graph are also converted into corresponding Cytoscape elements.

3. the `shownetwork` function initializes a dash application to visualize the network with Cytoscape. This function uses `make_cytoscape_elements` to create the Cytoscape elements and defines the layout and style definitions for the nodes and edges, including custom shapes for sources and sinks. The application also includes a button to export the visualization as an image.


### Installation requirements

oemof.solph > 0.5   
dash   
dash_cytoscape   

we tested this notebok with:  
 
oemof.solph==0.5.5   
dash==2.18.1   
dash_cytoscape==1.0.2   

#### Authors:

10.10.2024 - andreas.wunsch@iosb.fraunhofer.de , tobias.hoerter@iosb.fraunhofer.de

### Copyright and License

**Copyright**: oemof developer group  
**License**: GPLv3

In [1]:
from oemof.solph import (
    EnergySystem,
    Bus,
    Flow,
)
from oemof.solph.components import (
    Sink,
    Source,
    Converter,
    GenericStorage,
)
from oemof_visio import ESGraphRenderer


First create a dummy oemof energysystem without datetime index (not necessary):

In [2]:
energysystem = EnergySystem()

bgas = Bus(label="natural_gas")
bel = Bus(label="electricity")
grid_sink = Sink(label="excess_bel", inputs={bel: Flow()})
gas = Source(label="rgas",outputs={bgas: Flow()})
wind = Source(label="wind",outputs={bel: Flow()})
pv = Source(label="pv",outputs={bel: Flow()})
demand = Sink(label="demand",inputs={bel: Flow()},)
PowerPlant = Converter(label="pp_gas", inputs={bgas: Flow()}, outputs={bel: Flow()})
storage = GenericStorage(label="storage",inputs={bel: Flow()},outputs={bel: Flow()})

energysystem.add(bgas, bel, grid_sink, gas, wind, pv, demand, PowerPlant, storage)

Let's define the functions described above to visualize the energysystem with Dash:

In [3]:
import dash
from dash import html
import dash_cytoscape as cyto
import networkx as nx
from dash.dependencies import Input, Output


def make_network(energysystem):
    """
    Creates a network graph from an oemof energy system.

    Parameters:
    -----------
    energysystem : oemof.solph.EnergySystem
        An oemof energy system containing the nodes and edges of the network.

    Returns:
    --------
    G : networkx.DiGraph
        A directed graph representing the energy system. The nodes are the components
        of the energy system, and the edges represent the connections between the components.

    Description:
    -------------
    The function iterates over the nodes of the energy system and adds them to the graph.
    Based on the type of the component (Source, Sink, Converter, Bus, GenericStorage),
    colors and shapes are assigned, which can later be used in a Cytoscape model.
    The edges of the graph are created based on the input and output components of each node.
    """
    G = nx.DiGraph()  # Directed Graph using networkx
    
    # Iterate over model-components
    # for the shapes and colors that are later used in the cytoscape model we look at: https://js.cytoscape.org/#style/node-body
    # the goal is to visualize the components with the OEMOF symbols 

    for component in energysystem.nodes:
        G.add_node(component.label)  # use the real Oemof label for nodes
        if isinstance(component, Source):
            G.nodes[component.label]['color'] = 'yellow'
            G.nodes[component.label]['shape'] = 'custom-source'
        elif isinstance(component, Sink):
            G.nodes[component.label]['color'] = 'green'
            G.nodes[component.label]['shape'] = 'custom-sink'
        elif isinstance(component, Converter):
            G.nodes[component.label]['color'] = 'gray'
            G.nodes[component.label]['shape'] = 'rectangle'
        elif isinstance(component, Bus):
            G.nodes[component.label]['color'] = 'red'
            G.nodes[component.label]['shape'] = 'ellipse'
        elif isinstance(component, GenericStorage):
            G.nodes[component.label]['color'] = 'black'  
            G.nodes[component.label]['shape'] = 'round-rectangle'
        else:
            G.nodes[component.label]['color'] = 'light-blue'
            G.nodes[component.label]['shape'] = 'round-rectangle'

        # create edges based on input/ouputs of components
        for input_component in component.inputs:
            G.add_edge(input_component.label, component.label)  
        for output_component in component.outputs:
            G.add_edge(component.label, output_component.label)  
    
    return G

def make_cytoscape_elements(G):
    """
    Converts a NetworkX graph into Cytoscape elements for plotting with Dash.

    Parameters:
    -----------
    G : networkx.DiGraph
        A directed graph where nodes and edges represent the energy system components 
        and connections, respectively. Each node should have 'color' and 'shape' attributes.
    """
    nodes = [{'data': {'id': node,
                       'label': node,
                       'color': G.nodes[node]['color'],
                       'shape': G.nodes[node]['shape'],}} 
                       for node in G.nodes()]
    edges = [{'data': {'source': u, 'target': v}} for u, v in G.edges()]
    return nodes + edges
    
def shownetwork(network):
    """
    Initializes a Dash application to visualize a network using Cytoscape.

    Parameters:
    -----------
    network : networkx.DiGraph
        A directed graph where nodes and edges represent the energy system components 
        and connections, respectively. Each node should have 'color' and 'shape' attributes.

    Description:
    -------------
    This function sets up a Dash application to visualize the given network using Cytoscape.
    It uses the `make_cytoscape_elements` function to convert the NetworkX graph into Cytoscape
    elements. The layout and styling for the nodes and edges are defined, including custom shapes 
    for sources and sinks. The application includes a button to export the visualization as an image.
    """

    # Initialize dash app
    # check docs here: https://dash.plotly.com/cytoscape
    app = dash.Dash(__name__, suppress_callback_exceptions=True)  # suppress callback exceptions for callback ids that are not in layout
    
    # use the function from above to create cytoscape elements for dash
    elements = make_cytoscape_elements(network)
    
    # Layout for dash app 
    cyto.load_extra_layouts()  # dagre or klay are part of extra layouts, import is needed!
    app.layout = html.Div([
        cyto.Cytoscape(
            id='cytoscape',
            layout={'name': 'dagre'},  # dagre / klay are best suited to visualize the model
            style={'width': '100%', 'height': '500px'},
            elements=elements,  # use the cytoscape elements
            stylesheet=[
                {
                    'selector': 'node',
                    'style': {
                        'content': 'data(label)',  # uses OEMOF label 
                        'background-color': 'data(color)',  # and the assigned color for the corresponding OEMOF structure
                        'font-size': 12,
                        'color': 'white',
                        'text-valign': 'center',
                        'text-halign': 'center',
                        'width': '150px',
                        'height': '50px',
                        'shape': 'data(shape)',  # takes the shape properties from the model above
                    }
                },
                {
                    # since a trapezoid is not part of the standard shapes provided within the Cytoscape layout, they are created by using custom points to define the edges of a shape.
                    # To replicate the design used for OEMOF-structures, this needs to be done for the sources and the sinks
                    'selector': '[shape = "custom-source"]',
                    'style': {  # modify the source style here
                        'shape': 'polygon',
                        'shape-polygon-points': '-0.5 0.5, 0.5 0.5, 1 -0.5, -1 -0.5',  # define points for the edges of the trapezoid
                        'width': '200px',
                        'height': '100px',
                        'color': 'black'
                    }
                },
                {
                    'selector': '[shape = "custom-sink"]',
                    'style': {  # modify the sink style here
                        'shape': 'polygon',
                        'shape-polygon-points': '-0.5 -0.5, 0.5 -0.5, 1 0.5, -1 0.5',  # define points for edges of trapezoid
                        'width': '200px',
                        'height': '100px',
                    }
                },
                {
                    'selector': 'edge',
                    'style': {
                        'curve-style': 'straight',
                        'width': 2,
                        'line-color': 'gray',  # grey works better for dark themes
                        'target-arrow-color': 'gray',  # arrows to show direction of flows
                        'target-arrow-shape': 'triangle', 
                        'arrow-scale': 2,
                    }
                }
            ]
        ),
        # This part is only for downloading the image
        html.Button("Export as Image", id="btn-image", n_clicks=0)
    ])

    app.clientside_callback(
        """
        function(n_clicks) {
            console.log('Button clicked:', n_clicks);  // Debugging output
            if (n_clicks > 0) {
                var cy = window.cy;
                console.log('Cytoscape instance:', cy);  // Debugging output
                if (cy) {
                    var png64 = cy.png({scale: 3, full: true});  // Increase scale for higher resolution
                    var a = document.createElement('a');
                    a.href = png64;
                    a.download = 'Oemof_model.png';
                    a.click();
                } else {
                    console.error('Cytoscape instance is not defined');  // Debugging output
                }
            }
            return 'Export as Image';
        }
        """,
        Output('btn-image', 'children'),
        Input('btn-image', 'n_clicks')
    )
   
    # running the dash app
    app.run_server(mode='inline', debug=True)
    # Important sidenote: Should you encounter an error message, such as: 
    # "Address 'http://127.0.0.1:8050' already in use.
    # Try passing a different port to run_server."
    # This means that you are currently already running a dash application on the same port (maybe another file is open).
    # In the error message above, the port 8050 is being used. To bypass this, either restart the kernel and close the file, 
    # or (for example if you want to have multiple graphs open), you could manually change the port in the run_server command.
    # In this example it would be sufficient to swap to 8060:   
    # app.run_server(port=8060, mode='inline', debug=True)

Call the functions to visualize the energysystem:

In [4]:
network = make_network(energysystem)
shownetwork(network)